```python
## This is a Stock market perfomance analysis of the leading e- commerce companies publicly traded in the United States. 
```

```python
## To collect the real-time data needed for analysis, yahoo finance API(yfinance) has to be installed via pip.
```

In [47]:
import pandas as pd
import yfinance as yf
from datetime import datetime


In [48]:
start_date = datetime.now() - pd.DateOffset(months = 3)
end_date = datetime.now()
tickers = ['MELI','AMZN','EBAY','BABA','SHOP','JD']


In [49]:
df_list =[]

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker','Date'])
print(df.head())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                          Open         High          Low        Close  \
Ticker Date                                                             
MELI   2023-07-17  1128.300049  1181.300049  1128.300049  1179.170044   
       2023-07-18  1170.050049  1210.000000  1170.050049  1208.660034   
       2023-07-19  1219.030029  1232.579956  1210.939941  1225.750000   
       2023-07-20  1210.000000  1223.989990  1194.130005  1198.339966   
       2023-07-21  1209.219971  1228.920044  1201.989990  1221.030029   

                     Adj Close   Volume  
Ticker Date                     

```python
## In the code above, stock prices for six e-commerce companies for the last 3 months have been downloaded: MercadoLibre, Amazon, Ebay, Alibaba, Shopify and Jingdong Mall
```

In [50]:
df = df.reset_index()
print(df.head())

  Ticker       Date         Open         High          Low        Close  \
0   MELI 2023-07-17  1128.300049  1181.300049  1128.300049  1179.170044   
1   MELI 2023-07-18  1170.050049  1210.000000  1170.050049  1208.660034   
2   MELI 2023-07-19  1219.030029  1232.579956  1210.939941  1225.750000   
3   MELI 2023-07-20  1210.000000  1223.989990  1194.130005  1198.339966   
4   MELI 2023-07-21  1209.219971  1228.920044  1201.989990  1221.030029   

     Adj Close   Volume  
0  1179.170044   547200  
1  1208.660034   641800  
2  1225.750000   558100  
3  1198.339966   627700  
4  1221.030029  1116300  


```python
## Next, we use a line graph to look at the perfomance of the companies in the stock market.
```

In [51]:
import plotly.express as px
import statsmodels
fig = px.line(df, x = 'Date', y = 'Close', color = 'Ticker', title = "ECOMMERCE STOCK MARKET PRICES FOR THE LAST THREE MONTHS.")
fig.show()

```python
## Plotting an area chart to view similarities or differences of the stock prices easily
```

In [52]:
fig = px.area(df, x = 'Date',y='Close',color='Ticker',facet_col='Ticker',labels={'Date':'Date','Close':'Closing Price','Ticker':'Company'},title='Stock prices for Ecommerce Companies')
fig.show()


```python
## Using Moving Averages to identify patterns in price movement over the three months. MA10 and MA20 to be used.
```

In [53]:
df['MA10']=df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20']=df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Average for {ticker}')
    print(group[['MA10','MA20']])
    

Moving Average for AMZN
           MA10        MA20
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
69          NaN         NaN
..          ...         ...
125  127.192001  131.524001
126  127.777000  130.873000
127  128.412000  130.253500
128  128.678999  129.723500
129  128.987999  129.352000

[65 rows x 2 columns]
Moving Average for BABA
          MA10     MA20
195        NaN      NaN
196        NaN      NaN
197        NaN      NaN
198        NaN      NaN
199        NaN      NaN
..         ...      ...
255  85.537000  86.1745
256  85.695999  86.1815
257  85.592999  86.0070
258  85.320999  85.8545
259  85.098999  85.7190

[65 rows x 2 columns]
Moving Average for EBAY
          MA10     MA20
130        NaN      NaN
131        NaN      NaN
132        NaN      NaN
133        NaN      NaN
134        NaN      NaN
..         ...      ...
190  43.212000  43.5165
191  43.227999  43.4835
192  43.125999  43.3970
193  42.896000  43

```python
## Visualizing the Moving averages using line graphs.NOTE: MA10 crossing above MA20 indicates the price will continue rising. MA10 crossing below MA20 indicates the price will continue falling.
```

In [54]:
for ticker, group in df.groupby('Ticker'):
        fig=px.line(group,x='Date',y=['Close','MA10','MA20'], title=f"{ticker} Moving Averages")
        fig.show()
        

```python
## Finding out fluctuations of stock prices for the three months (commonly known as volatility). NOTE: High volatility means frequent price movements. Low volatility means less frequent price movements.
```

In [55]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop = True)
fig=px.line(df, x='Date',y='Volatility',color='Ticker',title='Volatility of Ecommerce Companies')
fig.show()


```python
## Finding out whether there is correlation between stock prices of two companies; Alibaba and Amazon.
```

In [56]:
Amazon = df.loc[df['Ticker']=='AMZN',['Date','Close']].rename(columns={'Close':'AMZN'})
Alibaba = df.loc[df['Ticker']=='BABA',['Date','Close']].rename(columns={'Close':'BABA'})
df_corr = pd.merge(Amazon, Alibaba, on='Date')


```python
## Creating a scatter plot to clearly see the correlation. NOTE: If stock prices of both companies rise/increase together, then it shows similarities between both companies hence a strong correlation.
```

In [57]:
fig=px.scatter(df_corr,x='AMZN',y='BABA',trendline='ols',title='Correlation between Amazon and Alibaba')
fig.show()